In [1]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as sig
import tqdm.notebook
import sycomore
from sycomore.units import *

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
# PCASL: A typical real sequence may use 750 0.5 ms, 20◦, Hann RF pulses over a 1500 ms period
labelling_length = 2200*ms # ms
flip_angle = 5*deg
gamma = 42.6e6 * Hz/T # Hz/T
Gamma = gamma * 2 * np.pi * rad # rads/Ts

dt = 0.05*ms
idle1, idle2 = 10*ms, 10*ms

r = np.linspace(-0.1, 0.1, 5) * mm
xx, yy = np.meshgrid(r, r)
voxel = np.zeros(xx.shape + (int(round((labelling_length + idle1 + idle2) / dt + 1)),3))

In [3]:
T1 = 1000*ms
T2 = 200*ms
zero = 0*mT/m
G = 2*mT/m
spoil = 400*mT/m
DeltaT = 0.1*ms

In [4]:
flowrate = 0.005 # cm/ms
flowffset = 1.25 # cm
def trajectory(t):
    """
    Return position Time in 
    """
    r = flowrate * t - flowffset
    return r

In [5]:
trajectory(0), trajectory(labelling_length.convert_to(ms))

(-1.25, 9.75)

In [6]:
aliased_labelling_plane = (1 / (gamma * G * DeltaT)).convert_to(cm)
aliased_labelling_plane

11.737089201877934

In [7]:
species = sycomore.Species(T1, T2)

pbar = tqdm.notebook.tqdm(total=xx.size)
for xind, x in enumerate(r):
    for yind, y in enumerate(r):
        pbar.update()

        idle = sycomore.bloch.time_interval(species, dt)
        pulse_20 = sycomore.bloch.pulse(flip_angle)

        t = 0*s
        M = np.array([0,0,1,1])

        record = [[t, M[:3]/M[3]]]
        
        length = idle1
        for _ in range(int(round(length / dt))):
            t = t+dt
            M = idle @ M
            record.append([t, M[:3]/M[3]])
        
        for i in range(int(round(labelling_length / DeltaT))):
            M = pulse_20 @ M
            # record.append([t, M[:3]/M[3]])

            length = DeltaT
            for _ in range(int(round(length / dt))):
                t = t+dt
                position = [x, y + trajectory(t.convert_to(ms)) * cm] # loop to sim full isochromat within a voxel -> can see echo and spoiler
                grad = sycomore.bloch.time_interval(species, dt, gradient_amplitude=[zero, G], position=position)
                M = grad @ M
                record.append([t, M[:3]/M[3]])

        length = idle2
        for _ in range(int(round(length / dt))):
            t = t+dt
            M = idle @ M
            record.append([t, M[:3]/M[3]])
        
        time, magnetization = list(zip(*record))
        magnetization = np.array(magnetization)
        voxel[xind, yind] = magnetization

  0%|          | 0/25 [00:00<?, ?it/s]

In [8]:
x_axis = [x.convert_to(ms) for x in time]

fig = plt.figure(figsize=(8, 6), dpi=100)
plt.plot(
    x_axis, np.linalg.norm(magnetization[:, :2], axis=-1), label="$M_\perp$")
plt.plot(x_axis, magnetization[:, 0], label="$M_x$")
plt.plot(x_axis, magnetization[:, 1], label="$M_y$")
plt.plot(x_axis, magnetization[:, 2], label="$M_z$")
plt.xlim(0)
plt.xlabel("Time (ms)")
plt.ylabel("$M/M_0$")
plt.legend()
plt.tight_layout()

In [9]:
transverse_mag = (voxel[..., 0] + 1j * voxel[..., 1]).mean(axis=(0, 1))
longitudinal_mag = voxel[..., 2].mean(axis=(0, 1))

In [10]:
longitudinal_mag.min()

-0.5518509633660504

In [11]:
x_axis[longitudinal_mag.argmin()]

314.34999999997353

In [23]:
x_axis = [x.convert_to(ms) for x in time]

fig = plt.figure(figsize=(8, 6), dpi=300)
plt.plot(x_axis, np.abs(transverse_mag), label="$M_\perp$", alpha=0.5)
# plt.plot(x_axis, transverse_mag.real, label="$M_x$")
plt.plot(x_axis, transverse_mag.imag, label="$M_y$", alpha=0.5)
plt.plot(x_axis, longitudinal_mag, label="$M_z$")
plt.xlim(left=0, right=x_axis[-1])
plt.ylim(bottom=-0.6, top=1.03)
plt.xlabel("Time (ms)")
plt.ylabel("$M/M_0$")
plt.legend()
plt.title('Linear Flow-Induced Adiabatic Inversion - PCASL Bloch Sim Magnetization vs Time')
plt.tight_layout()
plt.savefig('poster_figures/Linear Flow-Induced Adiabatic Inversion - PCASL Bloch Sim Magnetization vs Time')